In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-12-17 06:05:41.594151: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 06:05:41.684289: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 06:05:41.684422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 06:05:41.688625: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 06:05:41.704095: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 06:05:41.706241: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Assuming you have a dataset directory with 'covid' and 'non_covid' subdirectories containing X-ray images
dataset_directory = './models/datasets/c_dataset'

In [3]:
# Create an ImageDataGenerator to preprocess and augment the images
image_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [4]:
# Create train and validation generators
train_generator = image_generator.flow_from_directory(
    dataset_directory,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = image_generator.flow_from_directory(
    dataset_directory,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 145 images belonging to 2 classes.
Found 36 images belonging to 2 classes.


In [5]:
# Build a CNN model for COVID-19 prediction
covid_19_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),

    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification (0 or 1)
])

2023-12-17 06:05:53.790119: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 75759616 exceeds 10% of free system memory.
2023-12-17 06:05:53.837238: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 75759616 exceeds 10% of free system memory.
2023-12-17 06:05:53.876377: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 75759616 exceeds 10% of free system memory.


In [6]:
# model compiling using adam optimizer
covid_19_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Train the COVID-19 model
covid_19_model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10


2023-12-17 06:06:02.114758: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 75759616 exceeds 10% of free system memory.
2023-12-17 06:06:02.137161: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 75759616 exceeds 10% of free system memory.


5/5 [==============================] - 14s 2s/step - loss: 1.1161 - accuracy: 0.6345 - val_loss: 0.6009 - val_accuracy: 0.6389
Epoch 2/10
5/5 [==============================] - 9s 2s/step - loss: 0.4948 - accuracy: 0.7793 - val_loss: 0.3636 - val_accuracy: 0.8889
Epoch 3/10
5/5 [==============================] - 9s 2s/step - loss: 0.3494 - accuracy: 0.8276 - val_loss: 0.5422 - val_accuracy: 0.8056
Epoch 4/10
5/5 [==============================] - 10s 2s/step - loss: 0.3412 - accuracy: 0.8621 - val_loss: 0.4303 - val_accuracy: 0.7778
Epoch 5/10
5/5 [==============================] - 9s 2s/step - loss: 0.2620 - accuracy: 0.9103 - val_loss: 0.4841 - val_accuracy: 0.8056
Epoch 6/10
5/5 [==============================] - 9s 2s/step - loss: 0.1758 - accuracy: 0.9103 - val_loss: 0.2848 - val_accuracy: 0.8611
Epoch 7/10
5/5 [==============================] - 9s 2s/step - loss: 0.1852 - accuracy: 0.9172 - val_loss: 0.4879 - val_accuracy: 0.8056
Epoch 8/10
5/5 [==============================] - 

In [8]:
# saving the model
covid_19_model.save('./models/model_files/covid_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
